In [ ]:
# load the saved parameters and do visualization for W, a and C_alpha.
import numpy as np
import matplotlib.pyplot as plt
import pickle

path = '/data3/wangshaobo/ICL/RPE/results_paper/NGram/C_W_a_parent3_n10000_L100_S3_H5_1.0_1.0_1.0_optsgd_w+2.0_w-0.01_D3_c_alpha_init0.01_a_init0.01_alpha0.1_n-epochs[10, 10, 10].npz'
data = np.load(path)
print(data.keys())